## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos (versión completa)

---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP4. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [3]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [4]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]

## Dimensión: completitud

### (1a) Filas

In [5]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna_c = df_creditos.isna().sum()
nulos_x_columna_t = df_tarjetas.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_c}\n")
print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_t}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64

Cantidad de filas que tienen valores nulos por atributo:
id_cliente             0
antiguedad_cliente     0
estado_civil           0
estado_cliente         0
gastos_ult_12m         0
genero                 0
limite_credito_tc      0
nivel_educativo        0
nivel_tarjeta          0
operaciones_ult_12m    0
personas_a_cargo       0
dtype: int64


In [6]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :
0 (0.0)%


In [7]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_tarjetas.axes[1])

df_tarjetas['completitud_fila'] = (df_tarjetas.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_tarjetas[df_tarjetas['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :
0 (0.0)%


### (1b) Dataset

In [8]:
completitud_dc = df_creditos.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d] - creditos - :")
print(f"{completitud_dc} ({round((completitud_dc  / cantidad_filas_creditos) * 100, 2)})%\n")

completitud_dt = df_tarjetas.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :")
print(f"{completitud_dt} ({round((completitud_dt  / cantidad_filas_tarjetas) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d] - creditos - :
1225 (12.1)%

Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :
0 (0.0)%


----

## Dimensión: exactitud

### (2) Formato válido

In [13]:
# No se encuentran atributos con formato específico

----

## Dimensión: exactitud

### (3) Valores ajustados

Atributo: **edad**

In [14]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = df_creditos['edad'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['edad']) - df_creditos['edad'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
edad
22     2228
23     2190
24     1926
25     1646
26     1328
21      799
20        6
144       2
123       2
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
81,715623483.0,144,4800,3,4.0,ALQUILER,250000,INVERSIONES,0.02,13.57,0,N,0.000000
183,815500983.0,144,6000,2,4.0,HIPOTECA,200000,EDUCACIÓN,0.03,11.86,0,N,0.000000
575,717956283.0,123,20400,3,2.0,ALQUILER,80004,EDUCACIÓN,0.25,10.25,0,N,0.000000
747,712881483.0,123,20000,4,7.0,ALQUILER,78000,INVERSIONES,0.26,NaN,0,N,0.083333


Cantidad detectada: 4


In [15]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 4
Porcentaje de filas con errores de rango de valores (atributo edad): 0.04 %
Evaluación: no cumplimiento


Atributo: **Importe_Solicitado**

In [23]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: importe

valores2 = df_creditos['importe_solicitado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos2 = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos2 > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos2}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado2 = df_creditos[df_creditos['importe_solicitado'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado2) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado2.shape[0]}")

Distribución inicial del atributo: 
importe_solicitado
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: count, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila


Cantidad detectada: 0


Atributo: **duracion_credito**

In [17]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: duracion_credito

valores = df_creditos['duracion_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['duracion_credito'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
duracion_credito
2    3404
3    3364
4    3359
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila


Cantidad detectada: 0


Atributo: **antiguedad_empleado**

In [25]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_empleado

valores3 = df_creditos['antiguedad_empleado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos3 = len(df_creditos['antiguedad_empleado']) - df_creditos['antiguedad_empleado'].count() # Conteo de nulos

if cantidad_nulos3 > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos3}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado3 = df_creditos[df_creditos['antiguedad_empleado'] > 50]

print("Se visualizan las filas con errores de rango:")
display(resultado3) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado3.shape[0]}")

Distribución inicial del atributo: 
importe_solicitado
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: count, Length: 509, dtype: int64

Cantidad de nulos en el atributo: 337

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,0.0
210,715999608.0,21,20000,4,123.0,HIPOTECA,192000,INVERSIONES,0.10,6.54,0,N,0.0


Cantidad detectada: 2


In [26]:
def calcular_rangos_valores_antiguedad_empleado():
    antiguedad_empleado_valores_fuera_rango = resultado3.shape[0] + cantidad_nulos3
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {antiguedad_empleado_valores_fuera_rango}")
    
    indicador3 = (antiguedad_empleado_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador3 * 100, 2)} %")

    if (indicador3 > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_antiguedad_empleado()

Cantidad de filas con valores fuera de rango en atributo edad: 339
Porcentaje de filas con errores de rango de valores (atributo edad): 3.35 %
Evaluación: no cumplimiento


Atributo: **situacion_vivienda** 


In [28]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = df_creditos['situacion_vivienda'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['situacion_vivienda_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
situacion_vivienda
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


Atributo: **objetivo_credito** 

In [31]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: objetivo_credito

valores = df_creditos['objetivo_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'EDUCACIÓN|SALUD|INVERSIONES|PAGO_DEUDAS|PERSONAL|MEJORAS_HOGAR' 

# Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito_ok'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['objetivo_credito_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['objetivo_credito_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
objetivo_credito
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok


Cantidad detectada: 0


Atributo: **estado_credito** 

In [33]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_creditos['estado_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['estado_credito']) - df_creditos['estado_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = '1|0' # Se define una re de los valores validos según el anexo

df_creditos['estado_credito_ok'] = df_creditos['estado_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['estado_credito_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['estado_credito_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_credito
0    7635
1    2492
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,estado_credito_ok


Cantidad detectada: 0


Atributo: **falta_pago** 

In [34]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_creditos['falta_pago'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['falta_pago']) - df_creditos['falta_pago'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'Y|N' # Se define una re de los valores validos según el anexo

df_creditos['falta_pago_ok'] = df_creditos['falta_pago'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['falta_pago_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['falta_pago_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
falta_pago
N    8359
Y    1768
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,estado_credito_ok,falta_pago_ok


Cantidad detectada: 0


Atributo: **tasa_interes** 

In [40]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos


valores3 = df_creditos['tasa_interes'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos3 = len(df_creditos['tasa_interes']) - df_creditos['tasa_interes'].count() # Conteo de nulos

if cantidad_nulos3 > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos3}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado3 = df_creditos[df_creditos['tasa_interes'] > 70]

print("Se visualizan las filas con errores de rango:")
display(resultado3) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado3.shape[0]}")

Distribución inicial del atributo: 
falta_pago
N    8359
Y    1768
Name: count, dtype: int64

Cantidad de nulos en el atributo: 912

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,estado_credito_ok,falta_pago_ok


Cantidad detectada: 0


Atributo: **pct_ingreso** 

In [46]:
valores3 = df_creditos['pct_ingreso'].value_counts()  # Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo:\n{valores3}\n")

cantidad_nulos3 = len(df_creditos['pct_ingreso']) - df_creditos['pct_ingreso'].count()  # Conteo de nulos

if cantidad_nulos3 > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos3}\n")  # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Se identifica y cuenta a los valores que no cumplen la condición definida
resultado3 = df_creditos[(df_creditos['pct_ingreso'] <= 0) & (df_creditos['pct_ingreso'] >= 1)]

print("Se visualizan las filas con errores de rango:")
display(resultado3)  # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado3.shape[0]}")


Distribución inicial del atributo:
pct_ingreso
0.10    517
0.13    451
0.08    440
0.11    434
0.09    431
       ... 
0.67      1
0.58      1
0.72      1
0.68      1
0.70      1
Name: count, Length: 72, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,estado_credito_ok,falta_pago_ok


Cantidad detectada: 0


----

Tarjetas

Atributo: **antiguedad_cliente** 

In [48]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_cliente

valores = df_tarjetas['antiguedad_cliente'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['antiguedad_cliente']) - df_tarjetas['antiguedad_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['antiguedad_cliente'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
antiguedad_cliente
36.0    2463
37.0     358
34.0     353
38.0     347
39.0     341
40.0     333
31.0     318
35.0     317
33.0     305
30.0     300
41.0     297
32.0     289
28.0     275
43.0     273
42.0     271
29.0     241
44.0     230
45.0     227
27.0     206
46.0     197
26.0     186
47.0     171
25.0     165
48.0     162
24.0     160
49.0     141
23.0     116
22.0     105
56.0     103
50.0      96
21.0      83
51.0      80
53.0      78
20.0      74
13.0      70
19.0      63
52.0      62
18.0      58
54.0      53
55.0      42
17.0      39
15.0      34
16.0      29
14.0      16
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila


Cantidad detectada: 0


Atributo: **personas_a_cargo** 

In [49]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: personas_a_cargo

valores = df_tarjetas['personas_a_cargo'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['personas_a_cargo']) - df_tarjetas['personas_a_cargo'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['personas_a_cargo'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
personas_a_cargo
3.0    2732
2.0    2655
1.0    1838
4.0    1574
0.0     904
5.0     424
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila


Cantidad detectada: 0


Atributo: **estado_civil** 

In [50]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_civil

valores = df_tarjetas['estado_civil'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_civil']) - df_tarjetas['estado_civil'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'CASADO|SOLTERO|DESCONOCIDO|DIVORCIADO' # Se define una re de los valores validos según el anexo

df_tarjetas['estado_civil_ok'] = df_tarjetas['estado_civil'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['estado_civil_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['estado_civil_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_civil
CASADO         4687
SOLTERO        3943
DESCONOCIDO     749
DIVORCIADO      748
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok


Cantidad detectada: 0


Atributo: **estado_cliente** 

In [61]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_tarjetas['estado_cliente'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_cliente']) - df_tarjetas['estado_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ACTIVO|PASIVO' # Se define una re de los valores validos según el anexo

df_tarjetas['estado_cliente_ok'] = df_tarjetas['estado_cliente'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['estado_cliente_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['estado_cliente_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_cliente
ACTIVO    8500
PASIVO    1627
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok,estado_cliente_ok


Cantidad detectada: 0


Atributo: **genero** 

In [51]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_tarjetas['genero'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['genero']) - df_tarjetas['genero'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'F|M' # Se define una re de los valores validos según el anexo

df_tarjetas['genero_ok'] = df_tarjetas['genero'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['genero_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['genero_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
genero
F    5358
M    4769
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,genero_ok


Cantidad detectada: 0


Atributo: **nivel_educativo** 

In [52]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_tarjetas['nivel_educativo'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['nivel_educativo']) - df_tarjetas['nivel_educativo'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'SECUNDARIO_COMPLETO|UNIVERSITARIO_INCOMPLETO|UNIVERSITARIO_COMPLETO|POSGRADO_INCOMPLETO|POSGRADO_COMPLETO|DESCONOCIDO' # Se define una re de los valores validos según el anexo

df_tarjetas['nivel_educativo_ok'] = df_tarjetas['nivel_educativo'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['nivel_educativo_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['nivel_educativo_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
nivel_educativo
UNIVERSITARIO_COMPLETO      3128
UNIVERSITARIO_INCOMPLETO    2500
SECUNDARIO_COMPLETO         2013
DESCONOCIDO                 1519
POSGRADO_INCOMPLETO          516
POSGRADO_COMPLETO            451
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,genero_ok,nivel_educativo_ok


Cantidad detectada: 0


Atributo: **nivel_tarjeta** 

In [53]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_tarjetas['nivel_tarjeta'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['nivel_tarjeta']) - df_tarjetas['nivel_tarjeta'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'Blue|Gold|Silver|Platinum' # Se define una re de los valores validos según el anexo

df_tarjetas['nivel_tarjeta_ok'] = df_tarjetas['nivel_tarjeta'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['nivel_tarjeta_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['nivel_tarjeta_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
nivel_tarjeta
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok


Cantidad detectada: 0


Atributo: **limite_credito_tc** 

In [59]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

valores = df_tarjetas['limite_credito_tc'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['limite_credito_tc']) - df_tarjetas['limite_credito_tc'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['limite_credito_tc'] > 40000]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
limite_credito_tc
34516.0    508
1438.3     507
9959.0      18
15987.0     18
23981.0     12
          ... 
9173.0       1
8156.0       1
5853.0       1
5997.0       1
14657.0      1
Name: count, Length: 6205, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok


Cantidad detectada: 0


---

## Dimensión: consistencia

### (4) Claves únicas

Dataset: **datos_creditos**

In [62]:
#Se obtiene el valor de la cantidad de filas actual
cant_antes = df_creditos.shape[0] 

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_creditos.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_creditos.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_creditos.shape[0]

# Se imprimen ambos valores
print('Dataset: creditos')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: creditos
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


Dataset: **datos_tarjetas**

In [63]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [64]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila_y,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok,estado_cliente_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,SECUNDARIO_COMPLETO,Blue,111.0,3.0,0.0,True,True,True,True,True
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,DESCONOCIDO,Blue,21.0,0.0,0.0,True,True,True,True,True
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,POSGRADO_COMPLETO,Blue,23.0,3.0,0.0,True,True,True,True,True
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,UNIVERSITARIO_INCOMPLETO,Blue,82.0,2.0,0.0,True,True,True,True,True
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,UNIVERSITARIO_INCOMPLETO,Blue,59.0,2.0,0.0,True,True,True,True,True


Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [65]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [66]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila_y,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok,estado_cliente_ok,regla_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,Blue,58.0,4.0,0.0,True,True,True,True,True,err
1550,711232608.0,21,8000,4,3.0,ALQUILER,15500,MEJORAS_HOGAR,0.52,7.51,...,Blue,35.0,5.0,0.0,True,True,True,True,True,err
1900,711864783.0,24,10000,2,0.0,HIPOTECA,18000,PAGO_DEUDAS,0.56,12.18,...,Blue,32.0,1.0,0.0,True,True,True,True,True,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,Blue,45.0,3.0,0.0,True,True,True,True,True,err
2809,713555133.0,22,8000,4,3.0,ALQUILER,14400,MEJORAS_HOGAR,0.56,9.99,...,Blue,40.0,3.0,0.0,True,True,True,True,True,err


Cantidad de filas que no cumplen la regla: 15


In [67]:
def regla_pct_ingresos_duracion(row):
    duracion_minima = row.duracion_credito  # Duración mínima permitida
    pct_ingreso = row.pct_ingreso  # Porcentaje de ingresos
    importe_solicitado = row.importe_solicitado  # Importe solicitado
    situacion_vivienda = row.situacion_vivienda  # Situación de vivienda
    
    # Si la duración es mínima y el porcentaje de ingresos no excede el 60% o es propietario de su vivienda, es 'ok'
    if duracion_minima and (pct_ingreso <= 0.6 or situacion_vivienda == 'Propietario'):
        return 'ok'
    else:
        # Si no cumple la regla, es un error
        return 'err'

# Aplicar la función para todos los elementos del dataset
regla_pct_ingresos_duracion = df_integrado.apply(lambda row: regla_pct_ingresos_duracion(row), axis=1).rename("regla_pct_ingresos_duracion")

# Unir los resultados al dataset inicial
df_resultado_regla2 = pd.concat([df_integrado, regla_pct_ingresos_duracion], axis=1)

# Visualizar los datos que no cumplen la regla
print("Tuplas que no cumplen con la Regla 2:\n")
display(df_resultado_regla2[df_resultado_regla2.regla_pct_ingresos_duracion == 'err'].head())

# Verificar la cantidad de elementos que no cumplen la regla
aux_regla2 = df_resultado_regla2[df_resultado_regla2.regla_pct_ingresos_duracion == 'err']
print(f"Cantidad de filas que no cumplen la Regla 2: {aux_regla2.shape[0]}")


Tuplas que no cumplen con la Regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila_y,estado_civil_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok,estado_cliente_ok,regla_pct_ingresos_duracion
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,Blue,58.0,4.0,0.0,True,True,True,True,True,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,Blue,38.0,4.0,0.0,True,True,True,True,True,err
752,709616958.0,25,22750,4,3.0,HIPOTECA,32004,PERSONAL,0.71,6.17,...,Silver,73.0,2.0,0.0,True,True,True,True,True,err
1518,711173058.0,23,20000,4,3.0,ALQUILER,32000,PAGO_DEUDAS,0.63,12.61,...,Blue,29.0,2.0,0.0,True,True,True,True,True,err
1631,711374283.0,23,22250,3,2.0,ALQUILER,35000,PAGO_DEUDAS,0.64,12.69,...,Blue,32.0,3.0,0.0,True,True,True,True,True,err


Cantidad de filas que no cumplen la Regla 2: 23


Exportación del dataset integrado

In [68]:
df_integrado.to_csv("../../data/processed/datos_integrados.csv", sep=";", index=False)